In [142]:
from pandas import read_csv

from datasets.severstal_coco import DatasetCOCO
from torchvision import transforms
from models.clipseg import CLIPDensePredT
import torch
from PIL import Image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [143]:
data_path = '/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/train_subimages'
weights = '/home/eas/Enol/pycharm_projects/clipseg_steel_defect/logs/rd64-7K-vit16-cbh-coco-enol-5classes_no_neg/weights.pth'
mean = [0.34388125, 0.34388125, 0.34388125]
std = [0.13965334, 0.13965334, 0.13965334]
image_size = 256
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
COCO_CLASSES = {1: 'network of fine, hairline cracks or fissures on the surface of the steel',
                2: 'scale—oxides embedded into the steel plate',
                3: 'shallow, narrow grooves or lines on the surface of the steel',
                4: 'impurity or foreign material embedded within the steel matrix',
                5: 'defects on a steel plate'}
model = CLIPDensePredT(version='ViT-B/16', reduce_dim=64)
model.load_state_dict(torch.load(weights, weights_only=True, map_location=torch.device('cpu')), strict=False)
model.cuda()
model.eval()
split = 'train'

In [144]:
df = pd.DataFrame(columns = ['class_id', 'embedding'])

In [145]:
coco_dataset = DatasetCOCO(data_path, transform, split, True)

loading annotations into memory...
Done (t=1.04s)
creating index...
index created!


In [146]:
class_ids = coco_dataset.ids_by_class
metadata = coco_dataset.img_metadata

In [147]:
idx = 0
for i in class_ids:
    class_id = i
    for ann in tqdm(class_ids[class_id]):
        query = metadata.loadAnns(ids=ann)[0]
        image = Image.open(os.path.join(data_path, query['image_id']))
        q = transform(image).unsqueeze(0)
        q = q.cuda()
        visual_q, _, _ = model.visual_forward(q)
        df.loc[idx] = [class_id, visual_q.squeeze(0).cpu().numpy().tolist()]
        idx += 1


100%|██████████| 48985/48985 [13:30<00:00, 60.47it/s]


In [148]:
df.to_csv('../Severstal/train_embeddings.csv')

In [149]:
df.head()

,class_id,embedding
0,1,"[-0.38728979229927063, 0.1325313150882721, 0.10961241275072098, -0.0776892900466919, -0.2376573383808136, 0.396444708108902, 0.4293062090873718, -0.7315817475318909, 0.4360772669315338, -0.21442754566669464, 0.38084766268730164, -0.058034516870975494, -0.16634228825569153, -0.09514516592025757, -0.035816311836242676, -0.07400727272033691, -0.08032824844121933, -0.19801253080368042, -0.10912565886974335, 0.031279195100069046, 0.3076708912849426, 0.07886675000190735, 0.24258442223072052, -0.4971737265586853, -0.10252947360277176, 0.2857493758201599, 0.22882609069347382, 0.2271960973739624, 0.32311880588531494, 0.15770292282104492, 0.48655959963798523, -0.3500517010688782, -0.1687813401222229, -0.0450945645570755, -0.008440330624580383, 0.059139810502529144, 0.0031319819390773773, -0.08210553228855133, 0.39164549112319946, -0.4161904454231262, 0.23444592952728271, -0.1493268609046936, -0.06440040469169617, -0.515926718711853, -0.24705302715301514, 0.0696784183382988, -0.032016247510910034, 0.05659513175487518, -0.0454588308930397, 0.02564576268196106, -1.0520837306976318, 0.27455171942710876, -0.21860134601593018, -0.03460659459233284, -0.41712895035743713, -0.20225539803504944, 0.08583514392375946, 0.10130418837070465, -0.29573720693588257, 0.38771989941596985, -0.19859738647937775, 0.2832903563976288, 0.0646592304110527, 0.2973759174346924, 0.01853802800178528, -0.00280144065618515, -0.19019627571105957, -0.042342446744441986, 0.060694217681884766, 0.023326363414525986, 0.2721039056777954, 0.21078523993492126, 0.21950533986091614, 0.29747268557548523, 0.09350097179412842, 0.01789318025112152, 0.01065075397491455, 0.05529436469078064, 0.04901812970638275, -0.24137505888938904, 0.020862117409706116, 1.1850284337997437, -0.38424891233444214, 0.05827374383807182, -0.4147138297557831, 0.04594685882329941, 0.019365280866622925, 0.3230944573879242, -0.11523213982582092, 0.4022407829761505, -0.29988038539886475, 0.005788251757621765, 0.44290459156036377, -0.4417787194252014, 0.04298654571175575, -0.05820927023887634, 0.4811069369316101, 0.06407156586647034, 0.10631725937128067, 0.04234514757990837, ...]"
1,1,"[-0.2801094055175781, 0.15718357264995575, -0.04015563800930977, -0.09137618541717529, -0.458618700504303, 0.29957664012908936, 0.2690992057323456, -0.6623987555503845, 0.46521905064582825, 0.15326252579689026, 0.40036439895629883, -0.26047730445861816, -0.10022713243961334, 0.04868073761463165, 0.059188250452280045, 0.016262397170066833, -0.5134696960449219, 0.00570862740278244, -0.04755609482526779, 0.0433519184589386, 0.22405296564102173, 0.21138912439346313, 0.1691884696483612, -0.10479582846164703, 0.12893274426460266, 0.5140072107315063, 0.1602390557527542, 0.25570589303970337, 0.24100430309772491, 0.23052600026130676, 0.1857987642288208, -0.14518029987812042, 0.23109591007232666, 0.02023504301905632, 0.12283474206924438, -0.1041703149676323, -0.17244169116020203, -0.03495006263256073, 0.18781721591949463, -0.3569689393043518, 0.20010986924171448, -0.21559344232082367, -0.04534490406513214, -0.2725090980529785, -0.23522822558879852, -0.08121226727962494, 0.039956748485565186, -0.17639410495758057, -0.21518222987651825, 0.10395504534244537, -0.7462994456291199, 0.22589901089668274, -0.32542985677719116, 0.1859058141708374, -0.2912631034851074, -0.3586724102497101, 0.08585558086633682, 0.026710210368037224, -0.3648502826690674, 0.17597728967666626, -0.1491468846797943, 0.25386619567871094, 0.08484192937612534, 0.26596954464912415, -0.126654714345932, 0.3657137453556061, -0.01639195904135704, -0.3195495009422302, 0.11555413156747818, -0.14012359082698822, 0.02452421933412552, 0.2973252236843109, -0.10456810891628265, 0.22481457889080048, -0.1663247048854828, 0.2770361304283142, -0.15046796202659607, 0.3947797417640686, 0.08398444950580597, -0.2013736367225647, 0.00796101987361908, 1.3161910772323608, -0.5672309398651123, -0.15336467325687408, -0.137942835688591, 0.1709488034248352, 0.43361765146255493, 

In [150]:
len(df.loc[0]['embedding'])

512